<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Logistic_Regression%2CXGB%2CRandom_Forest%2CNaive_bayes%2CSVC%2CDeep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.decomposition import PCA

from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import time

from six import StringIO
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import MaxAbsScaler
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout



In [2]:
# Load in the data
df = pd.read_csv('/content/edited complete.csv',encoding='latin',nrows=100000)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


One hot encoding:

In [3]:
#Unique values of the columns
df['race_type\xa0'].unique()
df['course_type'].unique()
df['track_condition\xa0'].unique()
df['track_id'].unique()


array(['AQU', 'BEL', 'SAR'], dtype=object)

In [4]:
s=df[['race_type\xa0','course_type','track_condition\xa0','track_id']]

In [5]:
ohe=OneHotEncoder()
feature_array=ohe.fit_transform(df[['race_type\xa0','course_type','track_condition\xa0','track_id']]).toarray()

In [6]:
feature_labels=ohe.categories_
print(feature_labels)

[array(['ALW', 'AOC', 'CLM', 'SOC', 'SST', 'STK', 'STR', 'WCL'],
      dtype=object), array(['D', 'I', 'M', 'O', 'T'], dtype=object), array(['FM ', 'FT ', 'GD ', 'MY ', 'SY ', 'YL '], dtype=object), array(['AQU', 'BEL', 'SAR'], dtype=object)]


In [7]:
np.hstack(feature_labels)

array(['ALW', 'AOC', 'CLM', 'SOC', 'SST', 'STK', 'STR', 'WCL', 'D', 'I',
       'M', 'O', 'T', 'FM ', 'FT ', 'GD ', 'MY ', 'SY ', 'YL ', 'AQU',
       'BEL', 'SAR'], dtype=object)

In [8]:
feature_labels=np.hstack(feature_labels)
f=pd.DataFrame(feature_array,columns=feature_labels)
pd.concat([df,f],axis=1)

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,...,T,FM,FT,GD,MY,SY,YL,AQU,BEL,SAR
0,AQU,01-01-2019,9,6,73,40.672946,-73.827587,600,D,GD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AQU,01-01-2019,9,6,74,40.672990,-73.827568,600,D,GD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AQU,01-01-2019,9,6,63,40.672510,-73.827781,600,D,GD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AQU,01-01-2019,9,6,64,40.672553,-73.827762,600,D,GD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,AQU,01-01-2019,9,6,65,40.672596,-73.827742,600,D,GD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,BEL,04-05-2019,3,2,83,40.717702,-73.726057,600,D,SY,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99996,BEL,04-05-2019,3,2,84,40.717692,-73.726116,600,D,SY,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99997,BEL,04-05-2019,3,2,73,40.717738,-73.725438,600,D,SY,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99998,BEL,04-05-2019,3,2,74,40.717739,-73.725500,600,D,SY,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [9]:

df.drop(['race_date','track_id','course_type','track_condition\xa0','race_type\xa0','jockey\xa0','program_number\xa0'],axis=1,inplace=True)
df=pd.concat([df,f],axis=1)

In [10]:
df.isnull().any()


race_number            False
trakus_index           False
latitude               False
longitude              False
distance_id            False
run_up_distance        False
purse                  False
post_time              False
weight_carried         False
odds                   False
position_at_finish     False
ALW                    False
AOC                    False
CLM                    False
SOC                    False
SST                    False
STK                    False
STR                    False
WCL                    False
D                      False
I                      False
M                      False
O                      False
T                      False
FM                     False
FT                     False
GD                     False
MY                     False
SY                     False
YL                     False
AQU                    False
BEL                    False
SAR                    False
dtype: bool

In [11]:
#Removing the null values

df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

race_number            False
trakus_index           False
latitude               False
longitude              False
distance_id            False
run_up_distance        False
purse                  False
post_time              False
weight_carried         False
odds                   False
position_at_finish     False
ALW                    False
AOC                    False
CLM                    False
SOC                    False
SST                    False
STK                    False
STR                    False
WCL                    False
D                      False
I                      False
M                      False
O                      False
T                      False
FM                     False
FT                     False
GD                     False
MY                     False
SY                     False
YL                     False
AQU                    False
BEL                    False
SAR                    False
dtype: bool

In [12]:
!pip install movecolumn
import movecolumn as mc
mc.MoveToLast(df,'position_at_finish\xa0')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,race_number,trakus_index,latitude,longitude,distance_id,run_up_distance,purse,post_time,weight_carried,odds,...,FM,FT,GD,MY,SY,YL,AQU,BEL,SAR,position_at_finish
0,9,73,40.672946,-73.827587,600,48,25000,420,120,2090,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
1,9,74,40.672990,-73.827568,600,48,25000,420,120,2090,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
2,9,63,40.672510,-73.827781,600,48,25000,420,120,2090,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
3,9,64,40.672553,-73.827762,600,48,25000,420,120,2090,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
4,9,65,40.672596,-73.827742,600,48,25000,420,120,2090,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3,83,40.717702,-73.726057,600,48,36000,108,123,430,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6
99996,3,84,40.717692,-73.726116,600,48,36000,108,123,430,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6
99997,3,73,40.717738,-73.725438,600,48,36000,108,123,430,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6
99998,3,74,40.717739,-73.725500,600,48,36000,108,123,430,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,6


In [13]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

Principal Component Analysis

In [14]:
pca = PCA(n_components=10)
X = pca.fit_transform(X)


In [15]:
X.shape
X

array([[-4.12409442e+04,  4.90307797e+02, -1.90488233e+02, ...,
         3.62138130e+00, -2.40935018e-01,  6.24336721e-02],
       [-4.12409440e+04,  4.90305411e+02, -1.90298806e+02, ...,
         3.62147629e+00, -2.40762613e-01,  6.25168591e-02],
       [-4.12409466e+04,  4.90331654e+02, -1.92382498e+02, ...,
         3.62043128e+00, -2.42652946e-01,  6.16014170e-02],
       ...,
       [-3.02488298e+04, -1.21909653e+03, -4.43837041e+02, ...,
        -1.77374899e+00, -6.76265171e-01,  7.18685769e-01],
       [-3.02488296e+04, -1.21909892e+03, -4.43647614e+02, ...,
        -1.77365330e+00, -6.76130375e-01,  7.18767436e-01],
       [-3.02488293e+04, -1.21910130e+03, -4.43458188e+02, ...,
        -1.77355758e+00, -6.75996678e-01,  7.18849267e-01]])

Training and testing the data

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)


Standardising the X

In [17]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_train dimension= ', y_test.shape)

X_train dimension=  (80000, 10)
X_test dimension=  (20000, 10)
y_train dimension=  (80000,)
y_train dimension=  (20000,)


Logistic Regression 

In [19]:
#fitting the model
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
lm.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [20]:
#Predictions
y_pred=lm.predict(X_test)

In [21]:
#Classification report,accuracy score
result1 = classification_report(y_test, y_pred)
print(result1)
lm_score = accuracy_score(y_test,y_pred)
print(lm_score)

              precision    recall  f1-score   support

           1       0.32      0.44      0.37      1992
           2       0.27      0.15      0.19      2573
           3       0.37      0.36      0.36      2353
           4       0.28      0.38      0.32      2364
           5       0.32      0.31      0.32      2993
           6       0.22      0.35      0.27      3459
           7       0.37      0.19      0.25      1648
           8       0.54      0.29      0.37      1448
           9       0.00      0.00      0.00       639
          10       0.75      0.38      0.51       400
          11       0.75      1.00      0.86       131

    accuracy                           0.31     20000
   macro avg       0.38      0.35      0.35     20000
weighted avg       0.32      0.31      0.30     20000

0.308


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB Classifier:

In [22]:
#Input the parameters
xgb = XGBClassifier(n_estimators=10)


In [23]:
#training time
training_start = time.perf_counter()
xgb.fit(X_train, y_train)
training_end = time.perf_counter()

In [24]:
#predicting time
prediction_start = time.perf_counter()
preds1 = xgb.predict(X_test)
prediction_end = time.perf_counter()

In [25]:
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start

In [26]:

print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

Time consumed for training: 5.710
Time consumed for prediction: 0.07653 seconds


In [27]:
XGB_score=accuracy_score(y_test,preds1)
XGB_score

0.70595

Random Forest classifier

In [28]:
#Input the parameters
rfc = RandomForestClassifier(n_estimators=100)


In [29]:
#Training time
training_start = time.perf_counter()
rfc.fit(X_train, y_train)
training_end = time.perf_counter()

In [30]:
#predicting time
prediction_start = time.perf_counter()
preds2 = rfc.predict(X_test)
prediction_end = time.perf_counter()

In [31]:
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start

In [32]:

print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

Time consumed for training: 26.771 seconds
Time consumed for prediction: 0.27685 seconds


In [33]:
RFC_score=accuracy_score(y_test,preds2)
RFC_score

1.0

Support Vector Machine

In [34]:
#Input the parameters
svc = SVC()


In [35]:
#Training time
training_start = time.perf_counter()
svc.fit(X_train, y_train)
training_end = time.perf_counter()

In [36]:
#Testing time
prediction_start = time.perf_counter()
preds3 = svc.predict(X_test)
prediction_end = time.perf_counter()

In [37]:

svc_train_time = training_end-training_start
svc_prediction_time = prediction_end-prediction_start

In [38]:

print("Time consumed for training: %4.3f seconds" % (svc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (svc_prediction_time))

Time consumed for training: 113.558 seconds
Time consumed for prediction: 38.49170 seconds


In [39]:
SVC_score=accuracy_score(y_test,preds3)
SVC_score

0.9696

Naive Bayes

In [40]:
#Input the parameters 
gnb = GaussianNB()


In [41]:
#Training time
training_start = time.perf_counter()
gnb.fit(X_train, y_train)
training_end = time.perf_counter()

In [42]:
#Predicting time
prediction_start = time.perf_counter()
preds4 = gnb.predict(X_test)
prediction_end = time.perf_counter()

In [43]:
gnb_train_time = training_end-training_start
gnb_prediction_time = prediction_end-prediction_start

In [44]:
print("Time consumed for training: %4.3f seconds" % (gnb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (gnb_prediction_time))

Time consumed for training: 0.028 seconds
Time consumed for prediction: 0.02127 seconds


In [45]:
Naive_bayes_score=accuracy_score(y_test,preds4)
Naive_bayes_score

0.29355

KNeighbours Classifier:

In [46]:
#Input the parameters
knn = KNeighborsClassifier()

In [47]:
#Training time
training_start = time.perf_counter()
knn.fit(X_train, y_train)
training_end = time.perf_counter()

In [48]:
#Predicting Time
prediction_start = time.perf_counter()
preds5 = knn.predict(X_test)
prediction_end = time.perf_counter()

In [49]:
knn_train_time = training_end-training_start
knn_prediction_time = prediction_end-prediction_start

In [50]:
print("Time consumed for training: %4.3f seconds" % (knn_train_time))
print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

Time consumed for training: 0.125 seconds
Time consumed for prediction: 0.52405 seconds


In [51]:
KNN_score=accuracy_score(y_test,preds5)
KNN_score

1.0

Results of all models

In [52]:
results=pd.DataFrame({
    'Model': ['Logistic Regression',"XGB",'Random_Forest','SVC','Naive_bayes','KNN'],
    'Accuracy_Score': [lm_score,XGB_score,RFC_score,SVC_score,Naive_bayes_score,KNN_score]
})

results=results.sort_values(by='Accuracy_Score', ascending=False)
results=results.set_index('Model')
results


,Accuracy_Score
Model,
Random_Forest,1.00000
KNN,1.00000
SVC,0.96960
XGB,0.70595
Logistic Regression,0.30800
Naive_bayes,0.29355


K -Fold cross validation

We will do cross validation for SVC and XG boost since they are giving the best scores.

We do this to get a more realistic result by testing the performance for 10 different train and test datasets and averaging the results.

Cross validation ensures that the above result is not arbitary and gives a more reliable performance check.

SVC

In [53]:
from sklearn.model_selection import cross_val_score
svc = SVC()
scores=cross_val_score(svc,X_train,y_train,cv=2,scoring='accuracy')

In [54]:
print(scores)
print(scores.mean())
print(scores.std())


[0.94825  0.948275]
0.9482625
1.2499999999970868e-05


XG Boost


In [55]:
xgb_cv = XGBClassifier(n_estimators=100)
scores = cross_val_score(xgb_cv, X_train, y_train, cv=2, scoring = "accuracy")
print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.9988   0.995475]
Mean: 0.9971375
Standard Deviation: 0.0016625000000000112


Only SVC is better since the cross validation score is very high for XG boost

A high mean corresponds to a more stable performance and a low standard deviation corresponds to smaller range of results.

Feature importance:
We can consider viewing the features importance for XGB since it is quite optimal.

In [56]:
X=pd.DataFrame(X)
importances=pd.DataFrame({'features': X.columns,
                          'importance' : xgb.feature_importances_})

importances=importances.sort_values(by='importance', ascending=False)
#importances.set_index('features')
importances


,features,importance
8,8,0.159666
1,1,0.124201
2,2,0.116784
7,7,0.110827
9,9,0.107988
3,3,0.105440
5,5,0.096854
6,6,0.094212
0,0,0.084029
4,4,0.000000


In [57]:
#importances.plot.bar(X.columns,importances.importance)

Clearly we can see that 0,1,2,3,4 

Decision Tree:

In [58]:

f#rom sklearn.feature_extraction import DictVectorizer
#dv = DictVectorizer(sparse=False)
#X_train = dv.fit_transform(X)

#dt = DecisionTreeClassifier(max_depth=2)
#dt.fit(X_train, y_train)


,ALW,AOC,CLM,SOC,SST,STK,STR,WCL,D,I,...,T,FM,FT,GD,MY,SY,YL,AQU,BEL,SAR
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99996,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99997,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
99998,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [59]:
#preds6=dt.predict(X_test)
#accuracy_score(y_test,preds6)

In [60]:
#from sklearn.tree import export_graphviz
#dot_data = StringIO()
#export_graphviz(dt, out_file=dot_data,  
 #               filled=True, rounded=True,
  #              special_characters=True, feature_names = X.get_feature_names_out(),class_names=['0','1'])
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

Deep learning 

In [61]:
#Initialising the model
model=Sequential()
model.add(Dense(10,activation='relu',input_dim=10)) #input layer
model.add(Dropout(0.1))
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                110       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


In [62]:
#Applying optimizers and callbacks
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
callback=EarlyStopping(monitor='val_loss', min_delta=0.0001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False)

In [63]:
#Compiling the model
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
#Fitting the model
history=model.fit(X_train,y_train,batch_size=200, epochs=100,
                    validation_data = (X_test,y_test), verbose=1, callbacks=callback)

Epoch 1/100
400/400 [==============================] - 2s 3ms/step - loss: -0.8167 - accuracy: 0.0991 - val_loss: -2.0032 - val_accuracy: 0.0996
Epoch 2/100
400/400 [==============================] - 1s 3ms/step - loss: -3.4416 - accuracy: 0.1050 - val_loss: -5.2018 - val_accuracy: 0.0996
Epoch 3/100
400/400 [==============================] - 1s 3ms/step - loss: -7.4705 - accuracy: 0.1059 - val_loss: -10.3737 - val_accuracy: 0.0996
Epoch 4/100
400/400 [==============================] - 1s 3ms/step - loss: -13.9170 - accuracy: 0.1060 - val_loss: -18.4114 - val_accuracy: 0.0996
Epoch 5/100
400/400 [==============================] - 1s 3ms/step - loss: -23.6434 - accuracy: 0.1060 - val_loss: -30.0984 - val_accuracy: 0.0996
Epoch 6/100
400/400 [==============================] - 1s 3ms/step - loss: -37.0763 - accuracy: 0.1060 - val_loss: -45.6533 - val_accuracy: 0.0996
Epoch 7/100
400/400 [==============================] - 1s 3ms/step - loss: -54.7225 - accuracy: 0.1060 - val_loss: -65.5633

Among all the models, deep learning is the worst, 